<a href="https://colab.research.google.com/github/yogesh4344/EmergencyVehicleDetection/blob/main/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Prashant-AV/Qualcomm-DL-Hackathon.git

Cloning into 'Qualcomm-DL-Hackathon'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (10/10), 30.68 MiB | 21.13 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [3]:
import pandas as pd

train_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/train/train.csv')
train_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [4]:
test_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/test.csv')
test_data.head()
#train_data.shape

,image_names
0,1960.jpg
1,668.jpg
2,2082.jpg
3,808.jpg
4,1907.jpg


In [5]:
sample_data = pd.read_csv('/content/Qualcomm-DL-Hackathon/sample_submission.csv')
sample_data.head()
#train_data.shape

,image_names,emergency_or_not
0,1960.jpg,0
1,668.jpg,0
2,2082.jpg,0
3,808.jpg,0
4,1907.jpg,0


In [10]:
import zipfile
images_dir = '/content/Qualcomm-DL-Hackathon/train/images'
with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-1.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

with zipfile.ZipFile('/content/Qualcomm-DL-Hackathon/train/images part-2.zip', 'r') as zip_ref:
    zip_ref.extractall(images_dir)

In [13]:
import shutil
import os

# Source and destination directories
source_dir1 = '/content/Qualcomm-DL-Hackathon/train/images/images part-1'
source_dir2 = '/content/Qualcomm-DL-Hackathon/train/images/images part-2'
destination_dir = '/content/Qualcomm-DL-Hackathon/train/images'


def copy_files(source, destination):
    for filename in os.listdir(source):
        source_file = os.path.join(source, filename)
        destination_file = os.path.join(destination, filename)
        if os.path.isfile(source_file):
          shutil.copy(source_file, destination_dir)


# Copy files from 'images part-1'
copy_files(source_dir1, destination_dir)

# Copy files from 'images part-2'
copy_files(source_dir2, destination_dir)

print("Files copied successfully!")

Files copied successfully!


In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os

# Assuming your image folders are named '0' and '1'
class ImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name)

        # Check if there's a second column for labels
        if self.data.shape[1] > 1:
            label = self.data.iloc[idx, 1]
        else:
            label = 0  # or None, depending on how you want to handle it

        if self.transform:
            image = self.transform(image)
        return image, label

# Data transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# Create datasets and dataloaders
train_dataset = ImageDataset(csv_file='/content/Qualcomm-DL-Hackathon/train/train.csv',
                             root_dir='/content/Qualcomm-DL-Hackathon/train/images',
                             transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128) # Adjust input size based on image dimensions and pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)  # 2 output classes

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16) # Adjust input size based on image dimensions and pooling
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Training loop
num_epochs = 20 # Adjust as needed
for epoch in range(num_epochs):
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training finished.")

Epoch [1/20], Loss: 0.5642
Epoch [2/20], Loss: 0.7320
Epoch [3/20], Loss: 0.2231
Epoch [4/20], Loss: 0.6167
Epoch [5/20], Loss: 0.0968
Epoch [6/20], Loss: 0.1632
Epoch [7/20], Loss: 0.1797
Epoch [8/20], Loss: 0.0871
Epoch [9/20], Loss: 0.1420
Epoch [10/20], Loss: 0.0186
Epoch [11/20], Loss: 0.0049
Epoch [12/20], Loss: 0.0062
Epoch [13/20], Loss: 0.0045
Epoch [14/20], Loss: 0.0109
Epoch [15/20], Loss: 0.0029
Epoch [16/20], Loss: 0.0023
Epoch [17/20], Loss: 0.0075
Epoch [18/20], Loss: 0.0109
Epoch [19/20], Loss: 0.0076
Epoch [20/20], Loss: 0.0011
Training finished.


In [29]:
test_dataset = ImageDataset(csv_file='/content/Qualcomm-DL-Hackathon/test.csv',
                            root_dir='/content/Qualcomm-DL-Hackathon/train/images',
                            transform=transform)

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(test_dataset)

# Make predictions on the test data
predictions = []
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Create a submission dataframe
submission_df = pd.DataFrame({'image_names': test_data['image_names'], 'emergency_or_not': predictions})
submission_df.to_csv('submission.csv', index=False)

print("Predictions saved to submission.csv")

Predictions saved to submission.csv
